## 1. Код + импликатура с равными priors

In [2]:

import numpy as np
import pandas as pd


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon*self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        # Ваш код тут
        
        #Lit = rownorm(self.lexicon)
        #x1 = Lit.T
        S = self.speaker()
        x2 = S.T
        L = rownorm(x2)
        
        return rownorm(L*self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


# Core lexicon:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1,1,0],
    [0,0,1],
    [0,1,0] 
], index=msgs, columns=states)

print("="*70 + "\nEven priors and all-0 message costs\n")
basic_mod = RSA(lexicon=lex, prior=[0.33, 0.33, 0.3], costs=[0.0, 0.0, 0.0])

display_reference_game(basic_mod)

print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.00,1.0,0.0,0.0
glasses,0.00,0.0,1.0,0.0
moustache,0.00,1.0,0.0,0.0
prior,0.33,0.33,0.3,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
moustache,0.0,1.0,0.0



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.75,0.25,0.0
glasses,0.00,0.00,1.0
moustache,0.00,1.00,0.0


импликатура вычисляется, потому что каждое из выражений для разных states имеет разную вероятность (если прагматический слушающий слышит *шляпа*, он "вычисляет" r1, потому что 0.75 > 0.25)

## 2. Импликатура с другими priors

In [3]:
# with priors [0.1, 0.45, 0.45]
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1,1,0],
    [0,0,1],
    [0,1,0] 
], index=msgs, columns=states)

#print("="*70 + "\nEven priors and all-0 message costs\n")
new_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])

display_reference_game(new_mod)

print("\nLiteral listener")
display(new_mod.literal_listener())

print("\nPragmatic speaker")
display(new_mod.speaker())

print("\nPragmatic listener")
display(new_mod.listener())

,r1,r2,r3,costs
hat,1.0,1.0,0.0,0.0
glasses,0.0,0.0,1.0,0.0
moustache,0.0,1.0,0.0,0.0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.181818,0.818182,0.0
glasses,0.000000,0.000000,1.0
moustache,0.000000,1.000000,0.0



Pragmatic speaker


,hat,glasses,moustache
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
hat,0.330579,0.669421,0.0
glasses,0.000000,0.000000,1.0
moustache,0.000000,1.000000,0.0


кажется, что импликатура не вычисляется из-за высоких вероятностей у r2 (когда прагматический слушающий слышит *усы*, он "вычисляет" r2, и когда слышит *шляпа*, тоже)

## 3. Игры с двумя скалярными импликатурами

In [7]:
costs = [[0, 0, 0], [-5, 0, 0], [0, -5, 0], [0, 0, -5], [-10, 0, 0]]
priors = [[0.33, 0.33, 0.33], [0.6, 0.2, 0.2], [0.1, 0.15, 0.75], [0.3, 0.4, 0.3]]
alphas = [1, 4, 8]


msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']

# 1 игра -- со слайда 35 
lex1 = pd.DataFrame([
    [0,1,0],
    [1,1,0],
    [1,0,1] 
], index=msgs, columns=states)

# 2 игра -- со слайда 37

lex2 = pd.DataFrame([
    [1,1,0],
    [0,0,1],
    [0,1,1] 
], index=msgs, columns=states)

### 1 игра

In [9]:
mod1 = RSA(lexicon=lex1, prior=[0.33, 0.33, 0.33], costs=[0.0, 0.0, 0.0])

display_reference_game(mod1)

print("\nLiteral listener")
display(mod1.literal_listener())

print("\nPragmatic speaker")
display(mod1.speaker())

print("\nPragmatic listener")
display(mod1.listener())

,r1,r2,r3,costs
hat,0.00,1.0,0.0,0.0
glasses,1.00,1.0,0.0,0.0
moustache,1.00,0.0,1.0,0.0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
moustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,0.000000,0.500000,0.5
r2,0.666667,0.333333,0.0
r3,0.000000,0.000000,1.0



Pragmatic listener


,r1,r2,r3
hat,0.000000,1.0,0.000000
glasses,0.600000,0.4,0.000000
moustache,0.333333,0.0,0.666667


In [10]:
# цена высказывания
for c in costs:
    mod1 = RSA(lexicon=lex1, prior=[0.33, 0.33, 0.33], costs=c)

    display_reference_game(mod1)

    print("\nLiteral listener")
    display(mod1.literal_listener())

    print("\nPragmatic speaker")
    display(mod1.speaker())

    print("\nPragmatic listener")
    display(mod1.listener())
    print('='*40)

,r1,r2,r3,costs
hat,0.00,1.0,0.0,0
glasses,1.00,1.0,0.0,0
moustache,1.00,0.0,1.0,0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
moustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,0.000000,0.500000,0.5
r2,0.666667,0.333333,0.0
r3,0.000000,0.000000,1.0


...

,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
moustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,0.000000,0.500000,0.5
r2,0.000091,0.999909,0.0
r3,0.000000,0.000000,1.0



Pragmatic listener


,r1,r2,r3
hat,0.000000,1.000000,0.000000
glasses,0.333354,0.666646,0.000000
moustache,0.333333,0.000000,0.666667


до какого-то момента изменение цены высказывания не влияет на вычислимость импликатур; но когда цена становится слишком большой (например, -10), импликатура перестает вычисляться (так, в последнем случае и при *glasses*, и при *hat* вычисляется r2)

In [11]:
# исходные вероятности
for p in priors:
    mod1 = RSA(lexicon=lex1, prior=p, costs=[0.0, 0.0, 0.0])

    display_reference_game(mod1)

    print("\nLiteral listener")
    display(mod1.literal_listener())

    print("\nPragmatic speaker")
    display(mod1.speaker())

    print("\nPragmatic listener")
    display(mod1.listener())
    
    print('='*40)

,r1,r2,r3,costs
hat,0.00,1.0,0.0,0.0
glasses,1.00,1.0,0.0,0.0
moustache,1.00,0.0,1.0,0.0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
moustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,0.000000,0.500000,0.5
r2,0.666667,0.333333,0.0
r3,0.000000,0.000000,1.0


...

,r1,r2,r3
hat,0.000000,1.000000,0.0
glasses,0.428571,0.571429,0.0
moustache,0.500000,0.000000,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,0.000000,0.461538,0.538462
r2,0.636364,0.363636,0.000000
r3,0.000000,0.000000,1.000000



Pragmatic listener


,r1,r2,r3
hat,0.000000,1.000000,0.00
glasses,0.487685,0.512315,0.00
moustache,0.350000,0.000000,0.65


изменение априорных вероятностей часто приводит к тому, что импликатура не вычисляется (например, в случаях prior = [0.6, 0.2, 0.2] или prior = [0.3, 0.4, 0.3])

In [13]:
# оптимальность
for a in alphas:
    mod1 = RSA(lexicon=lex1, prior=[0.33, 0.33, 0.33], costs=[0.0, 0.0, 0.0], alpha = a)

    display_reference_game(mod1)

    print("\nLiteral listener")
    display(mod1.literal_listener())

    print("\nPragmatic speaker")
    display(mod1.speaker())

    print("\nPragmatic listener")
    display(mod1.listener())
    
    print('='*40)

,r1,r2,r3,costs
hat,0.00,1.0,0.0,0.0
glasses,1.00,1.0,0.0,0.0
moustache,1.00,0.0,1.0,0.0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
moustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,0.000000,0.500000,0.5
r2,0.666667,0.333333,0.0
r3,0.000000,0.000000,1.0


...

,r1,r2,r3
hat,0.0,1.0,0.0
glasses,0.5,0.5,0.0
moustache,0.5,0.0,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,0.000000,0.500000,0.5
r2,0.996109,0.003891,0.0
r3,0.000000,0.000000,1.0



Pragmatic listener


,r1,r2,r3
hat,0.000000,1.000000,0.000000
glasses,0.992278,0.007722,0.000000
moustache,0.333333,0.000000,0.666667


кажется, с увеличением альфы выведение импликатуры упрощается (разброс между значениями в строчках увеличивается)

### 2 игра

In [14]:
mod2 = RSA(lexicon=lex2, prior=[0.33, 0.33, 0.33], costs=[0.0, 0.0, 0.0])

display_reference_game(mod2)

print("\nLiteral listener")
display(mod2.literal_listener())

print("\nPragmatic speaker")
display(mod2.speaker())

print("\nPragmatic listener")
display(mod2.listener())

,r1,r2,r3,costs
hat,1.00,1.0,0.0,0.0
glasses,0.00,0.0,1.0,0.0
moustache,0.00,1.0,1.0,0.0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
moustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,1.0,0.000000,0.000000
r2,0.5,0.000000,0.500000
r3,0.0,0.666667,0.333333



Pragmatic listener


,r1,r2,r3
hat,0.666667,0.333333,0.0
glasses,0.000000,0.000000,1.0
moustache,0.000000,0.600000,0.4


In [16]:
# цена высказывания
for c in costs:
    mod2 = RSA(lexicon=lex2, prior=[0.33, 0.33, 0.33], costs=c)

    display_reference_game(mod2)

    print("\nLiteral listener")
    display(mod2.literal_listener())

    print("\nPragmatic speaker")
    display(mod2.speaker())

    print("\nPragmatic listener")
    display(mod2.listener())
    
    print('='*40)

,r1,r2,r3,costs
hat,1.00,1.0,0.0,0
glasses,0.00,0.0,1.0,0
moustache,0.00,1.0,1.0,0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
moustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,1.0,0.000000,0.000000
r2,0.5,0.000000,0.500000
r3,0.0,0.666667,0.333333


...

,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
moustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.000000,0.000000
r2,0.000045,0.000000,0.999955
r3,0.000000,0.666667,0.333333



Pragmatic listener


,r1,r2,r3
hat,0.999955,0.000045,0.000000
glasses,0.000000,0.000000,1.000000
moustache,0.000000,0.749991,0.250009


здесь импликатура не выводится только при изменении цены высказывания *glasses*

In [17]:
# исходные вероятности
for p in priors:
    mod2 = RSA(lexicon=lex2, prior=p, costs=[0.0, 0.0, 0.0])

    display_reference_game(mod2)

    print("\nLiteral listener")
    display(mod2.literal_listener())

    print("\nPragmatic speaker")
    display(mod2.speaker())

    print("\nPragmatic listener")
    display(mod2.listener())
    
    print('='*40)

,r1,r2,r3,costs
hat,1.00,1.0,0.0,0.0
glasses,0.00,0.0,1.0,0.0
moustache,0.00,1.0,1.0,0.0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
moustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,1.0,0.000000,0.000000
r2,0.5,0.000000,0.500000
r3,0.0,0.666667,0.333333


...

,r1,r2,r3
hat,0.428571,0.571429,0.000000
glasses,0.000000,0.000000,1.000000
moustache,0.000000,0.571429,0.428571



Pragmatic speaker


,hat,glasses,moustache
r1,1.0,0.0,0.0
r2,0.5,0.0,0.5
r3,0.0,0.7,0.3



Pragmatic listener


,r1,r2,r3
hat,0.6,0.400000,0.000000
glasses,0.0,0.000000,1.000000
moustache,0.0,0.689655,0.310345


импликатура не выводится при prior = [0.1, 0.15, 0.75]

In [18]:
# исходные вероятности
for a in alphas:
    mod2 = RSA(lexicon=lex2, prior=[0.33, 0.33, 0.33], costs=[0.0, 0.0, 0.0], alpha = a)

    display_reference_game(mod2)

    print("\nLiteral listener")
    display(mod2.literal_listener())

    print("\nPragmatic speaker")
    display(mod2.speaker())

    print("\nPragmatic listener")
    display(mod2.listener())
    
    print('='*40)

,r1,r2,r3,costs
hat,1.00,1.0,0.0,0.0
glasses,0.00,0.0,1.0,0.0
moustache,0.00,1.0,1.0,0.0
prior,0.33,0.33,0.33,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
moustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,1.0,0.000000,0.000000
r2,0.5,0.000000,0.500000
r3,0.0,0.666667,0.333333


...

,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
moustache,0.0,0.5,0.5



Pragmatic speaker


,hat,glasses,moustache
r1,1.0,0.000000,0.000000
r2,0.5,0.000000,0.500000
r3,0.0,0.996109,0.003891



Pragmatic listener


,r1,r2,r3
hat,0.666667,0.333333,0.000000
glasses,0.000000,0.000000,1.000000
moustache,0.000000,0.992278,0.007722


здесь импликатуры выводятся при любом значении альфа (как и в предыдущей игре, разбросы стали больше)

в целом, изменение цены высказывания и априорных распределений часто затрудняют выведение скалярных импликатур, а увеличение параметра альфа -- упрощают

## 4. Другой феномен

Можно применить RSA к использованию идиом. Большая роль будет у исходных вероятностей (у некоторых идиом крайне низкая или нулевая вероятность буквального прочтения, например *raining cats and dogs*); также возможно нужен какой-нибудь параметр, отвечающий за контекст (например, *give a hand* может иметь как буквальное, так и идиоматическое прочтение (помочь) в разных контекстах)